# 00 – MPS Setup & Environment Check

Sanity check to verify that environment is ready for mechanistic interpretability work on Apple Silicon.

**Sections:**
1. MPS availability check
2. Load GPT-2 small
3. Single forward pass
4. Capture one activation and print its shape

## 1. MPS Availability Check

In [1]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"MPS available: {torch.backends.mps.is_available()}")
print(f"MPS built: {torch.backends.mps.is_built()}")

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"\nUsing device: {device}")

Python: 3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 08:06:15) [Clang 14.0.6 ]
PyTorch: 2.2.2
MPS available: True
MPS built: True

Using device: mps


## 2. Load GPT-2 Small

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()

print(f"Model: {model_name}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Device: {next(model.parameters()).device}")

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model: gpt2
Parameters: 124,439,808
Device: mps:0


## 3. Single Forward Pass

In [3]:
text = "One step at a time"
inputs = tokenizer(text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

print(f"Input tokens: {inputs['input_ids'].shape}")
print(f"Logits shape: {outputs.logits.shape}")  # [batch, seq_len, vocab_size]

Input tokens: torch.Size([1, 5])
Logits shape: torch.Size([1, 5, 50257])


## 4. Capture One Activation

Hook into layer 6 MLP output to verify we can extract internal activations.
This is the foundation of mechanistic interpretability: you can hook any layer/module to inspect what the model computes internally, enabling activation patching, probing, or SAE training.

In [4]:
activations = {}

def hook_fn(module, inp, out):
    activations["mlp_out"] = out.detach()

# Hook layer 6 MLP
handle = model.transformer.h[6].mlp.register_forward_hook(hook_fn)

with torch.no_grad():
    _ = model(**inputs)

handle.remove()

print(f"MLP output shape: {activations['mlp_out'].shape}")  # [batch, seq_len, hidden_dim=768]
print("\n✓ Setup complete! Ready for mechanistic interpretability.")

MLP output shape: torch.Size([1, 5, 768])

✓ Setup complete! Ready for mechanistic interpretability.
